# Finding similar documents with Word2Vec and Soft Cosine Measure 

Soft Cosine Measure (SCM) is a promising new tool in machine learning that allows us to submit a query and return the most relevant documents. In **part 1**, we will show how you can compute SCM between two documents using the `inner_product` method. In **part 2**, we will use `SoftCosineSimilarity` to retrieve documents most similar to a query and compare the performance against other similarity measures.

First, however, we go through the basics of what Soft Cosine Measure is.

## Soft Cosine Measure basics

Soft Cosine Measure (SCM) is a method that allows us to assess the similarity between two documents in a meaningful way, even when they have no words in common. It uses a measure of similarity between words, which can be derived [2] using [word2vec][] [3] vector embeddings of words. It has been shown to outperform many of the state-of-the-art methods in the semantic text similarity task in the context of community question answering [2].

[word2vec]: https://radimrehurek.com/gensim/models/word2vec.html

SCM is illustrated below for two very similar sentences. The sentences have no words in common, but by modeling synonymy, SCM is able to accurately measure the similarity between the two sentences. The method also uses the bag-of-words vector representation of the documents (simply put, the word's frequencies in the documents). The intution behind the method is that we compute standard cosine similarity assuming that the document vectors are expressed in a non-orthogonal basis, where the angle between two basis vectors is derived from the angle between the word2vec embeddings of the corresponding words.

![Soft Cosine Measure](soft_cosine_tutorial.png)

This method was perhaps first introduced in the article “Soft Measure and Soft Cosine Measure: Measure of Features in Vector Space Model” by Grigori Sidorov, Alexander Gelbukh, Helena Gomez-Adorno, and David Pinto ([link to PDF](http://www.scielo.org.mx/pdf/cys/v18n3/v18n3a7.pdf)).

In this tutorial, we will learn how to use Gensim's SCM functionality, which consists of the `inner_product` method for one-off computation, and the `SoftCosineSimilarity` class for corpus-based similarity queries.

> **Note**:
>
> If you use this software, please consider citing [1] and [2].
>

## Running this notebook
You can download this [Jupyter notebook](http://jupyter.org/), and run it on your own computer, provided you have installed the `gensim`, `jupyter`, `sklearn`, `pyemd`, and `wmd` Python packages.

The notebook was run on an Ubuntu machine with an Intel core i7-6700HQ CPU 3.10GHz (4 cores) and 16 GB memory. Assuming all resources required by the notebook have already been downloaded, running the entire notebook on this machine takes about 30 minutes.

In [1]:
# Initialize logging.
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## Part 1: Computing the Soft Cosine Measure

To use SCM, we need some word embeddings first of all. You could train a [word2vec][] (see tutorial [here](http://rare-technologies.com/word2vec-tutorial/)) model on some corpus, but we will use pre-trained word2vec embeddings.

[word2vec]: https://radimrehurek.com/gensim/models/word2vec.html

Let's create some sentences to compare.

In [2]:
sentence_obama = 'Obama speaks to the media in Illinois'.lower().split()
sentence_president = 'The president greets the press in Chicago'.lower().split()
sentence_orange = 'Having a tough time finding an orange juice press machine?'.lower().split()

The first two sentences have very similar content, and as such the SCM should be large. Before we compute the SCM, we want to remove stopwords ("the", "to", etc.), as these do not contribute a lot to the information in the sentences.

In [3]:
# Import and download stopwords from NLTK.
from nltk.corpus import stopwords
from nltk import download
download('stopwords')  # Download stopwords list.

# Remove stopwords.
stop_words = stopwords.words('english')
sentence_obama = [w for w in sentence_obama if w not in stop_words]
sentence_president = [w for w in sentence_president if w not in stop_words]
sentence_orange = [w for w in sentence_orange if w not in stop_words]

# Prepare a dictionary and a corpus.
from gensim import corpora
documents = [sentence_obama, sentence_president, sentence_orange]
dictionary = corpora.Dictionary(documents)

# Convert the sentences into bag-of-words vectors.
sentence_obama = dictionary.doc2bow(sentence_obama)
sentence_president = dictionary.doc2bow(sentence_president)
sentence_orange = dictionary.doc2bow(sentence_orange)

/mnt/storage/home/novotny/.virtualenvs/gensim/lib/python3.4/importlib/_bootstrap.py:321: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/mnt/storage/home/novotny/.virtualenvs/gensim/lib/python3.4/importlib/_bootstrap.py:321: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/novotny/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


2018-09-11 22:02:01,041 : INFO : 'pattern' package not found; tag filters are not available for English
2018-09-11 22:02:01,044 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-09-11 22:02:01,045 : INFO : built Dictionary(14 unique tokens: ['speaks', 'illinois', 'greets', 'juice', 'chicago']...) from 3 documents (total 15 corpus positions)


Now, as we mentioned earlier, we will be using some downloaded pre-trained embeddings. Note that the embeddings we have chosen here require a lot of memory. We will use the embeddings to construct a term similarity matrix that will be used by the `inner_product` method.

In [4]:
%%time
import gensim.downloader as api
from gensim.models import WordEmbeddingSimilarityIndex
from gensim.similarities import SparseTermSimilarityMatrix

w2v_model = api.load("glove-wiki-gigaword-50")
similarity_index = WordEmbeddingSimilarityIndex(w2v_model)
similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary)

2018-09-11 22:02:01,236 : INFO : loading projection weights from /home/novotny/gensim-data/glove-wiki-gigaword-50/glove-wiki-gigaword-50.gz
2018-09-11 22:02:26,984 : INFO : loaded (400000, 50) matrix from /home/novotny/gensim-data/glove-wiki-gigaword-50/glove-wiki-gigaword-50.gz
2018-09-11 22:02:26,985 : INFO : constructing a sparse term similarity matrix using <gensim.models.keyedvectors.WordEmbeddingSimilarityIndex object at 0x7f8d6e8615c0>
2018-09-11 22:02:26,986 : INFO : iterating over columns in dictionary order
2018-09-11 22:02:26,987 : INFO : PROGRESS: at 7.14% columns (1 / 14, 7.142857% density, 7.142857% projected density)
2018-09-11 22:02:26,988 : INFO : precomputing L2-norms of word weight vectors
/mnt/storage/home/novotny/.virtualenvs/gensim/lib/python3.4/site-packages/gensim/matutils.py:738: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if n

CPU times: user 27.8 s, sys: 2.43 s, total: 30.3 s
Wall time: 26.2 s


Let's compute SCM using the `inner_product` method.

In [5]:
similarity = similarity_matrix.inner_product(sentence_obama, sentence_president, normalized=True)
print('similarity = %.4f' % similarity)

similarity = 0.3790


Let's try the same thing with two completely unrelated sentences. Notice that the similarity is smaller.

In [6]:
similarity = similarity_matrix.inner_product(sentence_obama, sentence_orange, normalized=True)
print('similarity = %.4f' % similarity)

similarity = 0.1108


## Part 2: Similarity queries using `SoftCosineSimilarity`
You can use SCM to get the most similar documents to a query, using the `SoftCosineSimilarity` class. Its interface is similar to what is described in the [Similarity Queries](https://radimrehurek.com/gensim/tut3.html) Gensim tutorial.

### Qatar Living unannotated dataset
Contestants solving the community question answering task in the [SemEval 2016][semeval16] and [2017][semeval17] competitions had an unannotated dataset of 189,941 questions and 1,894,456 comments from the [Qatar Living][ql] discussion forums. As our first step, we will use the same dataset to build a corpus.

[semeval16]: http://alt.qcri.org/semeval2016/task3/
[semeval17]: http://alt.qcri.org/semeval2017/task3/
[ql]: http://www.qatarliving.com/forum

In [7]:
%%time
from itertools import chain
import json
from re import sub
from os.path import isfile

import gensim.downloader as api
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from nltk import download


download("stopwords")  # Download stopwords list.
stopwords = set(stopwords.words("english"))

def preprocess(doc):
    doc = sub(r'<img[^<>]+(>|$)', " image_token ", doc)
    doc = sub(r'<[^<>]+(>|$)', " ", doc)
    doc = sub(r'\[img_assist[^]]*?\]', " ", doc)
    doc = sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', " url_token ", doc)
    return [token for token in simple_preprocess(doc, min_len=0, max_len=float("inf")) if token not in stopwords]

corpus = list(chain(*[
    chain(
        [preprocess(thread["RelQuestion"]["RelQSubject"]), preprocess(thread["RelQuestion"]["RelQBody"])],
        [preprocess(relcomment["RelCText"]) for relcomment in thread["RelComments"]])
    for thread in api.load("semeval-2016-2017-task3-subtaskA-unannotated")]))

print("Number of documents: %d" % len(documents))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/novotny/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Number of documents: 3
CPU times: user 2min 37s, sys: 1.62 s, total: 2min 39s
Wall time: 2min 39s


Using the corpus we have just build, we will now construct a [dictionary][], a [TF-IDF model][tfidf], a [word2vec model][word2vec], and a term similarity matrix.

[dictionary]: https://radimrehurek.com/gensim/corpora/dictionary.html
[tfidf]: https://radimrehurek.com/gensim/models/tfidfmodel.html
[word2vec]: https://radimrehurek.com/gensim/models/word2vec.html

In [8]:
%%time
from multiprocessing import cpu_count

from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models import Word2Vec
from gensim.models import WordEmbeddingSimilarityIndex
from gensim.similarities import SparseTermSimilarityMatrix

dictionary = Dictionary(corpus)
tfidf = TfidfModel(dictionary=dictionary)
w2v_model = Word2Vec(corpus, workers=cpu_count(), min_count=5, size=300, seed=12345)
similarity_index = WordEmbeddingSimilarityIndex(w2v_model.wv)
similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf, nonzero_limit=100)

2018-09-11 22:05:07,212 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-09-11 22:05:07,485 : INFO : adding document #10000 to Dictionary(20088 unique tokens: ['authours', 'chimney', 'bombard', 'euro', 'lies']...)
2018-09-11 22:05:07,751 : INFO : adding document #20000 to Dictionary(29692 unique tokens: ['biscit', 'pharaonic', 'authours', 'chimney', 'unanimous']...)
2018-09-11 22:05:08,036 : INFO : adding document #30000 to Dictionary(37971 unique tokens: ['biscit', 'pharaonic', 'authours', 'chimney', 'unanimous']...)
2018-09-11 22:05:08,293 : INFO : adding document #40000 to Dictionary(43930 unique tokens: ['biscit', 'chimney', 'strangers', 'untruths', 'apes']...)
2018-09-11 22:05:08,550 : INFO : adding document #50000 to Dictionary(49340 unique tokens: ['biscit', 'chimney', 'strangers', 'zimbabawe', 'untruths']...)
2018-09-11 22:05:08,825 : INFO : adding document #60000 to Dictionary(54734 unique tokens: ['biscit', 'chimney', 'strangers', 'zimbabawe', 'smartness'].

2018-09-11 22:05:22,111 : INFO : adding document #550000 to Dictionary(195951 unique tokens: ['bladder', 'appreciat', 'pples', 'adib', 'strangers']...)
2018-09-11 22:05:22,389 : INFO : adding document #560000 to Dictionary(197956 unique tokens: ['bladder', 'appreciat', 'pples', 'adib', 'strangers']...)
2018-09-11 22:05:22,669 : INFO : adding document #570000 to Dictionary(200145 unique tokens: ['bladder', 'appreciat', 'pples', 'adib', 'strangers']...)
2018-09-11 22:05:22,926 : INFO : adding document #580000 to Dictionary(201859 unique tokens: ['bladder', 'appreciat', 'pples', 'adib', 'strangers']...)
2018-09-11 22:05:23,178 : INFO : adding document #590000 to Dictionary(203724 unique tokens: ['bladder', 'appreciat', 'pples', 'adib', 'strangers']...)
2018-09-11 22:05:23,438 : INFO : adding document #600000 to Dictionary(205607 unique tokens: ['bladder', 'appreciat', 'pples', 'adib', 'strangers']...)
2018-09-11 22:05:23,703 : INFO : adding document #610000 to Dictionary(207387 unique tok

2018-09-11 22:05:36,423 : INFO : adding document #1090000 to Dictionary(291139 unique tokens: ['bladder', 'appreciat', 'pples', 'adib', 'strangers']...)
2018-09-11 22:05:36,695 : INFO : adding document #1100000 to Dictionary(293838 unique tokens: ['bladder', 'appreciat', 'pples', 'adib', 'strangers']...)
2018-09-11 22:05:36,955 : INFO : adding document #1110000 to Dictionary(295273 unique tokens: ['bladder', 'appreciat', 'pples', 'adib', 'strangers']...)
2018-09-11 22:05:37,219 : INFO : adding document #1120000 to Dictionary(296816 unique tokens: ['bladder', 'appreciat', 'pples', 'adib', 'strangers']...)
2018-09-11 22:05:37,478 : INFO : adding document #1130000 to Dictionary(298552 unique tokens: ['bladder', 'appreciat', 'pples', 'adib', 'strangers']...)
2018-09-11 22:05:37,692 : INFO : adding document #1140000 to Dictionary(299628 unique tokens: ['bladder', 'appreciat', 'pples', 'adib', 'strangers']...)
2018-09-11 22:05:37,957 : INFO : adding document #1150000 to Dictionary(301139 uni

2018-09-11 22:05:50,846 : INFO : adding document #1630000 to Dictionary(372956 unique tokens: ['pples', 'adib', 'strangers', 'kolayaalee', 'softpoint']...)
2018-09-11 22:05:51,107 : INFO : adding document #1640000 to Dictionary(374282 unique tokens: ['pples', 'adib', 'strangers', 'kolayaalee', 'softpoint']...)
2018-09-11 22:05:51,382 : INFO : adding document #1650000 to Dictionary(375746 unique tokens: ['pples', 'adib', 'strangers', 'kolayaalee', 'softpoint']...)
2018-09-11 22:05:51,646 : INFO : adding document #1660000 to Dictionary(377073 unique tokens: ['pples', 'adib', 'strangers', 'kolayaalee', 'softpoint']...)
2018-09-11 22:05:51,910 : INFO : adding document #1670000 to Dictionary(378393 unique tokens: ['pples', 'adib', 'strangers', 'kolayaalee', 'softpoint']...)
2018-09-11 22:05:52,170 : INFO : adding document #1680000 to Dictionary(379812 unique tokens: ['pples', 'adib', 'strangers', 'kolayaalee', 'softpoint']...)
2018-09-11 22:05:52,433 : INFO : adding document #1690000 to Dic

2018-09-11 22:06:04,891 : INFO : adding document #2160000 to Dictionary(449397 unique tokens: ['pples', 'adib', 'strangers', 'kolayaalee', 'softpoint']...)
2018-09-11 22:06:05,175 : INFO : adding document #2170000 to Dictionary(450649 unique tokens: ['pples', 'adib', 'strangers', 'kolayaalee', 'softpoint']...)
2018-09-11 22:06:05,438 : INFO : adding document #2180000 to Dictionary(451840 unique tokens: ['pples', 'adib', 'strangers', 'kolayaalee', 'softpoint']...)
2018-09-11 22:06:05,714 : INFO : adding document #2190000 to Dictionary(453020 unique tokens: ['pples', 'adib', 'strangers', 'kolayaalee', 'softpoint']...)
2018-09-11 22:06:05,981 : INFO : adding document #2200000 to Dictionary(454160 unique tokens: ['pples', 'adib', 'strangers', 'kolayaalee', 'softpoint']...)
2018-09-11 22:06:06,248 : INFO : adding document #2210000 to Dictionary(455302 unique tokens: ['pples', 'adib', 'strangers', 'kolayaalee', 'softpoint']...)
2018-09-11 22:06:06,532 : INFO : adding document #2220000 to Dic

...

2018-09-11 22:09:25,990 : INFO : PROGRESS: at 45.59% columns (211001 / 462807, 0.000283% density, 0.000364% projected density)
2018-09-11 22:09:26,033 : INFO : PROGRESS: at 45.81% columns (212001 / 462807, 0.000284% density, 0.000363% projected density)
2018-09-11 22:09:26,075 : INFO : PROGRESS: at 46.02% columns (213001 / 462807, 0.000284% density, 0.000363% projected density)
2018-09-11 22:09:26,147 : INFO : PROGRESS: at 46.24% columns (214001 / 462807, 0.000284% density, 0.000363% projected density)
2018-09-11 22:09:26,236 : INFO : PROGRESS: at 46.46% columns (215001 / 462807, 0.000284% density, 0.000363% projected density)
2018-09-11 22:09:26,292 : INFO : PROGRESS: at 46.67% columns (216001 / 462807, 0.000284% density, 0.000362% projected density)
2018-09-11 22:09:26,422 : INFO : PROGRESS: at 46.89% columns (217001 / 462807, 0.000285% density, 0.000363% projected density)
2018-09-11 22:09:26,465 : INFO : PROGRESS: at 47.10% columns (218001 / 462807, 0.000285% density, 0.000363% pro

2018-09-11 22:09:34,785 : INFO : PROGRESS: at 59.64% columns (276001 / 462807, 0.000322% density, 0.000393% projected density)
2018-09-11 22:09:34,977 : INFO : PROGRESS: at 59.85% columns (277001 / 462807, 0.000323% density, 0.000394% projected density)
2018-09-11 22:09:35,220 : INFO : PROGRESS: at 60.07% columns (278001 / 462807, 0.000324% density, 0.000395% projected density)
2018-09-11 22:09:35,501 : INFO : PROGRESS: at 60.28% columns (279001 / 462807, 0.000325% density, 0.000397% projected density)
2018-09-11 22:09:35,960 : INFO : PROGRESS: at 60.50% columns (280001 / 462807, 0.000327% density, 0.000400% projected density)
2018-09-11 22:09:36,200 : INFO : PROGRESS: at 60.72% columns (281001 / 462807, 0.000328% density, 0.000401% projected density)
2018-09-11 22:09:36,493 : INFO : PROGRESS: at 60.93% columns (282001 / 462807, 0.000330% density, 0.000403% projected density)
2018-09-11 22:09:36,720 : INFO : PROGRESS: at 61.15% columns (283001 / 462807, 0.000331% density, 0.000404% pro

2018-09-11 22:10:01,510 : INFO : PROGRESS: at 73.68% columns (341001 / 462807, 0.000448% density, 0.000531% projected density)
2018-09-11 22:10:02,327 : INFO : PROGRESS: at 73.90% columns (342001 / 462807, 0.000452% density, 0.000535% projected density)
2018-09-11 22:10:03,267 : INFO : PROGRESS: at 74.11% columns (343001 / 462807, 0.000456% density, 0.000540% projected density)
2018-09-11 22:10:03,968 : INFO : PROGRESS: at 74.33% columns (344001 / 462807, 0.000459% density, 0.000543% projected density)
2018-09-11 22:10:04,741 : INFO : PROGRESS: at 74.55% columns (345001 / 462807, 0.000463% density, 0.000547% projected density)
2018-09-11 22:10:05,494 : INFO : PROGRESS: at 74.76% columns (346001 / 462807, 0.000466% density, 0.000551% projected density)
2018-09-11 22:10:06,487 : INFO : PROGRESS: at 74.98% columns (347001 / 462807, 0.000471% density, 0.000556% projected density)
2018-09-11 22:10:07,188 : INFO : PROGRESS: at 75.19% columns (348001 / 462807, 0.000474% density, 0.000559% pro

2018-09-11 22:17:04,903 : INFO : PROGRESS: at 87.73% columns (406001 / 462807, 0.001926% density, 0.002166% projected density)
2018-09-11 22:17:13,755 : INFO : PROGRESS: at 87.94% columns (407001 / 462807, 0.001954% density, 0.002193% projected density)
2018-09-11 22:17:22,668 : INFO : PROGRESS: at 88.16% columns (408001 / 462807, 0.001983% density, 0.002220% projected density)
2018-09-11 22:17:31,544 : INFO : PROGRESS: at 88.37% columns (409001 / 462807, 0.002011% density, 0.002247% projected density)
2018-09-11 22:17:40,258 : INFO : PROGRESS: at 88.59% columns (410001 / 462807, 0.002037% density, 0.002271% projected density)
2018-09-11 22:17:49,178 : INFO : PROGRESS: at 88.81% columns (411001 / 462807, 0.002065% density, 0.002298% projected density)
2018-09-11 22:17:57,836 : INFO : PROGRESS: at 89.02% columns (412001 / 462807, 0.002090% density, 0.002321% projected density)
2018-09-11 22:18:06,604 : INFO : PROGRESS: at 89.24% columns (413001 / 462807, 0.002117% density, 0.002346% pro

CPU times: user 4h 38min 32s, sys: 4h 24min 33s, total: 9h 3min 5s
Wall time: 20min 43s


### Evaluation
Next, we will load the validation and test datasets that were used by the SemEval 2016 and 2017 contestants. The datasets contain 208 original questions posted by the forum members. For each question, there is a list of 10 threads with a human annotation denoting whether or not the thread is relevant to the original question. Our task will be to order the threads so that relevant threads rank above irrelevant threads.

In [9]:
datasets = api.load("semeval-2016-2017-task3-subtaskBC")

Finally, we will perform an evaluation to compare three unsupervised similarity measures – the Soft Cosine Measure, two different implementations of the [Word Mover's Distance][wmd], and standard cosine similarity. We will use the [Mean Average Precision (MAP)][map] as an evaluation measure and 10-fold cross-validation to get an estimate of the variance of MAP for each similarity measure.

[wmd]: http://vene.ro/blog/word-movers-distance-in-python.html
[map]: https://medium.com/@pds.bangalore/mean-average-precision-abd77d0b9a7e

In [10]:
from math import isnan
from time import time

from gensim.similarities import MatrixSimilarity, WmdSimilarity, SoftCosineSimilarity
import numpy as np
from sklearn.model_selection import KFold
from wmd import WMD

def produce_test_data(dataset):
    for orgquestion in datasets[dataset]:
        query = preprocess(orgquestion["OrgQSubject"]) + preprocess(orgquestion["OrgQBody"])
        documents = [
            preprocess(thread["RelQuestion"]["RelQSubject"]) + preprocess(thread["RelQuestion"]["RelQBody"])
            for thread in orgquestion["Threads"]]
        relevance = [
            thread["RelQuestion"]["RELQ_RELEVANCE2ORGQ"] in ("PerfectMatch", "Relevant")
            for thread in orgquestion["Threads"]]
        yield query, documents, relevance

def cossim(query, documents):
    # Compute cosine similarity between the query and the documents.
    query = tfidf[dictionary.doc2bow(query)]
    index = MatrixSimilarity(
        tfidf[[dictionary.doc2bow(document) for document in documents]],
        num_features=len(dictionary))
    similarities = index[query]
    return similarities

def softcossim(query, documents):
    # Compute Soft Cosine Measure between the query and the documents.
    query = tfidf[dictionary.doc2bow(query)]
    index = SoftCosineSimilarity(
        tfidf[[dictionary.doc2bow(document) for document in documents]],
        similarity_matrix)
    similarities = index[query]
    return similarities

def wmd_gensim(query, documents):
    # Compute Word Mover's Distance as implemented in PyEMD by William Mayner
    # between the query and the documents.
    index = WmdSimilarity(documents, w2v_model)
    similarities = index[query]
    return similarities

def wmd_relax(query, documents):
    # Compute Word Mover's Distance as implemented in WMD by Source{d}
    # between the query and the documents.
    words = [word for word in set(chain(query, *documents)) if word in w2v_model.wv]
    indices, words = zip(*sorted((
        (index, word) for (index, _), word in zip(dictionary.doc2bow(words), words))))
    query = dict(tfidf[dictionary.doc2bow(query)])
    query = [
        (new_index, query[dict_index])
        for new_index, dict_index in enumerate(indices)
        if dict_index in query]
    documents = [dict(tfidf[dictionary.doc2bow(document)]) for document in documents]
    documents = [[
        (new_index, document[dict_index])
        for new_index, dict_index in enumerate(indices)
        if dict_index in document] for document in documents]
    embeddings = np.array([w2v_model.wv[word] for word in words], dtype=np.float32)
    nbow = dict(((index, list(chain([None], zip(*document)))) for index, document in enumerate(documents)))
    nbow["query"] = tuple([None] + list(zip(*query)))
    distances = WMD(embeddings, nbow, vocabulary_min=1).nearest_neighbors("query")
    similarities = [-distance for _, distance in sorted(distances)]
    return similarities

strategies = {
    "cossim" : cossim,
    "softcossim": softcossim,
    "wmd-gensim": wmd_gensim,
    "wmd-relax": wmd_relax}

def evaluate(split, strategy):
    # Perform a single round of evaluation.
    results = []
    start_time = time()
    for query, documents, relevance in split:
        similarities = strategies[strategy](query, documents)
        assert len(similarities) == len(documents)
        precision = [
            (num_correct + 1) / (num_total + 1) for num_correct, num_total in enumerate(
                num_total for num_total, (_, relevant) in enumerate(
                    sorted(zip(similarities, relevance), reverse=True)) if relevant)]
        average_precision = np.mean(precision) if precision else 0.0
        results.append(average_precision)
    return (np.mean(results) * 100, time() - start_time)

def crossvalidate(args):
    # Perform a cross-validation.
    dataset, strategy = args
    test_data = np.array(list(produce_test_data(dataset)))
    kf = KFold(n_splits=10)
    samples = []
    for _, test_index in kf.split(test_data):
        samples.append(evaluate(test_data[test_index], strategy))
    return (np.mean(samples, axis=0), np.std(samples, axis=0))

In [11]:
%%time
from multiprocessing import Pool

args_list = [
    (dataset, technique)
    for dataset in ("2016-test", "2017-test")
    for technique in ("softcossim", "wmd-gensim", "wmd-relax", "cossim")]
with Pool() as pool:
    results = pool.map(crossvalidate, args_list)

2018-09-11 22:25:55,395 : INFO : precomputing L2-norms of word weight vectors
2018-09-11 22:25:55,423 : INFO : creating matrix with 10 documents and 462807 features
/mnt/storage/home/novotny/.virtualenvs/gensim/lib/python3.4/site-packages/gensim/matutils.py:738: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
2018-09-11 22:25:55,444 : INFO : Vocabulary size: 35 500
2018-09-11 22:25:55,447 : INFO : WCD
2018-09-11 22:25:55,450 : INFO : 0.0
2018-09-11 22:25:55,451 : INFO : First K WMD
2018-09-11 22:25:55,457 : INFO : creating matrix with 10 documents and 462807 features
/mnt/storage/home/novotny/.virtualenvs/gensim/lib/python3.4/site-packages/gensim/matutils.py:738: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np

2018-09-11 22:25:55,803 : INFO : Vocabulary size: 3 500
2018-09-11 22:25:55,804 : INFO : WCD
2018-09-11 22:25:55,807 : INFO : 0.0
2018-09-11 22:25:55,808 : INFO : First K WMD
2018-09-11 22:25:55,813 : INFO : creating matrix with 10 documents and 462807 features
2018-09-11 22:25:55,815 : INFO : [(-23.323942184448242, 1), (-23.062299728393555, 2), (-23.259197235107422, 4), (-22.928569793701172, 8), (-22.76712989807129, 7), (-22.777070999145508, 6), (-22.7784423828125, 3), (-21.306692123413086, 0), (-22.706424713134766, 5), (-20.88772201538086, 9)]
2018-09-11 22:25:55,817 : INFO : 0.0
2018-09-11 22:25:55,818 : INFO : P&P
2018-09-11 22:25:55,819 : INFO : creating matrix with 10 documents and 462807 features
2018-09-11 22:25:55,820 : INFO : stopped by early_stop condition
2018-09-11 22:25:55,822 : INFO : Vocabulary size: 21 500
2018-09-11 22:25:55,824 : INFO : WCD
2018-09-11 22:25:55,826 : INFO : 0.0
2018-09-11 22:25:55,828 : INFO : First K WMD
2018-09-11 22:25:55,841 : INFO : [(-21.3825092

2018-09-11 22:25:56,066 : INFO : [(-17.6803035736084, 9), (-16.80386734008789, 7), (-15.549836158752441, 0), (-15.82524299621582, 3), (-16.073200225830078, 6), (-14.176229476928711, 1), (-15.330875396728516, 4), (-15.516229629516602, 2), (-15.30439567565918, 5), (-15.97109317779541, 8)]
2018-09-11 22:25:56,067 : INFO : 0.0
2018-09-11 22:25:56,068 : INFO : P&P
2018-09-11 22:25:56,069 : INFO : stopped by early_stop condition
2018-09-11 22:25:56,075 : INFO : creating matrix with 10 documents and 462807 features
2018-09-11 22:25:56,079 : INFO : Vocabulary size: 12 500
2018-09-11 22:25:56,081 : INFO : WCD
2018-09-11 22:25:56,084 : INFO : 0.0
2018-09-11 22:25:56,085 : INFO : First K WMD
2018-09-11 22:25:56,091 : INFO : Vocabulary size: 7 500
2018-09-11 22:25:56,093 : INFO : WCD
2018-09-11 22:25:56,095 : INFO : 0.0
2018-09-11 22:25:56,095 : INFO : [(-21.149734497070312, 3), (-21.115989685058594, 9), (-20.278905868530273, 4), (-20.617753982543945, 7), (-21.0872859954834, 1), (-17.8920364379882

2018-09-11 22:25:56,310 : INFO : 0.0
2018-09-11 22:25:56,311 : INFO : P&P
2018-09-11 22:25:56,312 : INFO : stopped by early_stop condition
2018-09-11 22:25:56,313 : INFO : creating matrix with 10 documents and 462807 features
2018-09-11 22:25:56,316 : INFO : Vocabulary size: 18 500
2018-09-11 22:25:56,318 : INFO : WCD
2018-09-11 22:25:56,320 : INFO : 0.0
2018-09-11 22:25:56,321 : INFO : First K WMD
2018-09-11 22:25:56,332 : INFO : [(-20.602643966674805, 4), (-18.422197341918945, 7), (-19.27498435974121, 5), (-17.853513717651367, 1), (-18.41405487060547, 2), (-18.674968719482422, 9), (-17.420278549194336, 3), (-16.40138053894043, 6), (-0.0, 0), (-16.361116409301758, 8)]
2018-09-11 22:25:56,333 : INFO : 0.0
2018-09-11 22:25:56,335 : INFO : P&P
2018-09-11 22:25:56,335 : INFO : Vocabulary size: 12 500
2018-09-11 22:25:56,336 : INFO : stopped by early_stop condition
2018-09-11 22:25:56,336 : INFO : WCD
2018-09-11 22:25:56,338 : INFO : 0.0
2018-09-11 22:25:56,339 : INFO : First K WMD
2018-09

2018-09-11 22:25:56,630 : INFO : P&P
2018-09-11 22:25:56,631 : INFO : stopped by early_stop condition
2018-09-11 22:25:56,633 : INFO : creating matrix with 10 documents and 462807 features
2018-09-11 22:25:56,642 : INFO : [(-20.412126541137695, 1), (-19.901622772216797, 6), (-18.32648468017578, 9), (-17.38874053955078, 3), (-15.66531753540039, 0), (-15.41468334197998, 8), (-16.87537384033203, 5), (-16.159189224243164, 7), (-16.56991195678711, 2), (-0.0, 4)]
2018-09-11 22:25:56,643 : INFO : 0.0
2018-09-11 22:25:56,644 : INFO : P&P
2018-09-11 22:25:56,645 : INFO : stopped by early_stop condition
2018-09-11 22:25:56,652 : INFO : Vocabulary size: 10 500
2018-09-11 22:25:56,653 : INFO : creating matrix with 10 documents and 462807 features
2018-09-11 22:25:56,653 : INFO : WCD
2018-09-11 22:25:56,655 : INFO : 0.0
2018-09-11 22:25:56,656 : INFO : First K WMD
2018-09-11 22:25:56,662 : INFO : creating matrix with 10 documents and 462807 features
2018-09-11 22:25:56,664 : INFO : [(-20.3319644927

...

2018-09-11 22:28:22,254 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-09-11 22:28:22,255 : INFO : built Dictionary(42 unique tokens: ['resign', 'give', 'cover', 'number', 'cash']...) from 2 documents (total 62 corpus positions)
2018-09-11 22:28:22,268 : INFO : Removed 1 and 1 OOV words from document 1 and 2 (respectively).
2018-09-11 22:28:22,269 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-09-11 22:28:22,270 : INFO : built Dictionary(26 unique tokens: ['qatar', 'open', 'edge', 'thanks', 'roughly']...) from 2 documents (total 33 corpus positions)
2018-09-11 22:28:22,277 : INFO : Removed 0 and 1 OOV words from document 1 and 2 (respectively).
2018-09-11 22:28:22,278 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-09-11 22:28:22,279 : INFO : built Dictionary(45 unique tokens: ['work', 'open', 'c', 'better', 'get']...) from 2 documents (total 63 corpus positions)
2018-09-11 22:28:22,293 : INFO : Removed 1 and 1 OOV words fro

2018-09-11 22:28:25,950 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-09-11 22:28:25,951 : INFO : built Dictionary(31 unique tokens: ['know', 'thanks', 'arrival', 'would', 'like']...) from 2 documents (total 48 corpus positions)
2018-09-11 22:28:25,962 : INFO : precomputing L2-norms of word weight vectors
2018-09-11 22:28:27,649 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-09-11 22:28:27,650 : INFO : built Dictionary(30 unique tokens: ['entry', 'qa', 'need', 'month', 'also']...) from 2 documents (total 46 corpus positions)
2018-09-11 22:28:27,660 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-09-11 22:28:27,661 : INFO : built Dictionary(32 unique tokens: ['get', 'right', 'long', 'july', 'travel']...) from 2 documents (total 47 corpus positions)
2018-09-11 22:28:27,671 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-09-11 22:28:27,672 : INFO : built Dictionary(24 unique tokens: ['get', 'thanks', 'arriv

2018-09-11 22:28:33,135 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-09-11 22:28:33,136 : INFO : built Dictionary(37 unique tokens: ['wife', 'qatar', 'follow', 'tell', 'civil']...) from 2 documents (total 60 corpus positions)
2018-09-11 22:28:33,153 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-09-11 22:28:33,154 : INFO : built Dictionary(41 unique tokens: ['ask', 'wife', 'qatar', 'would', 'follow']...) from 2 documents (total 72 corpus positions)
2018-09-11 22:28:33,173 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-09-11 22:28:33,175 : INFO : built Dictionary(31 unique tokens: ['wife', 'qatar', 'follow', 'hello', 'son']...) from 2 documents (total 47 corpus positions)
2018-09-11 22:28:33,186 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-09-11 22:28:33,187 : INFO : built Dictionary(29 unique tokens: ['wife', 'qatar', 'follow', 'hello', 'son']...) from 2 documents (total 42 corpus positions)
2018-09

2018-09-11 22:28:38,734 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-09-11 22:28:38,736 : INFO : built Dictionary(43 unique tokens: ['info', 'poll', 'living', 'people', 'weeks']...) from 2 documents (total 61 corpus positions)
2018-09-11 22:28:38,752 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-09-11 22:28:38,753 : INFO : built Dictionary(25 unique tokens: ['open', 'customer', 'thanks', 'soon', 'please']...) from 2 documents (total 39 corpus positions)
2018-09-11 22:28:38,760 : INFO : Removed 1 and 0 OOV words from document 1 and 2 (respectively).
2018-09-11 22:28:38,761 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-09-11 22:28:38,762 : INFO : built Dictionary(29 unique tokens: ['open', 'customer', 'thanks', 'roughly', 'stll']...) from 2 documents (total 43 corpus positions)
2018-09-11 22:28:38,770 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-09-11 22:28:38,771 : INFO : built Dictionary(39 unique

CPU times: user 2.14 s, sys: 5.08 s, total: 7.22 s
Wall time: 2min 51s


The table below shows the pointwise estimates of means and standard variances for MAP scores and elapsed times. Baselines and winners for each year are displayed in bold. We can see that the Soft Cosine Measure gives a strong performance on both the 2016 and the 2017 dataset.

In [12]:
from IPython.display import display, Markdown

output = []
baselines = [
    (("2016-test", "**Winner (UH-PRHLT-primary)**"), ((76.70, 0), (0, 0))),
    (("2016-test", "**Baseline 1 (IR)**"), ((74.75, 0), (0, 0))),
    (("2016-test", "**Baseline 2 (random)**"), ((46.98, 0), (0, 0))),
    (("2017-test", "**Winner (SimBow-primary)**"), ((47.22, 0), (0, 0))),
    (("2017-test", "**Baseline 1 (IR)**"), ((41.85, 0), (0, 0))),
    (("2017-test", "**Baseline 2 (random)**"), ((29.81, 0), (0, 0)))]
table_header = ["Dataset | Strategy | MAP score | Elapsed time (sec)", ":---|:---|:---|---:"]
for row, ((dataset, technique), ((mean_map_score, mean_duration), (std_map_score, std_duration))) \
        in enumerate(sorted(chain(zip(args_list, results), baselines), key=lambda x: (x[0][0], -x[1][0][0]))):
    if row % (len(strategies) + 3) == 0:
        output.extend(chain(["\n"], table_header))
    map_score = "%.02f ±%.02f" % (mean_map_score, std_map_score)
    duration = "%.02f ±%.02f" % (mean_duration, std_duration) if mean_duration else ""
    output.append("%s|%s|%s|%s" % (dataset, technique, map_score, duration))

display(Markdown('\n'.join(output)))



Dataset | Strategy | MAP score | Elapsed time (sec)
:---|:---|:---|---:
2016-test|softcossim|77.15 ±10.83|4.48 ±0.56
2016-test|**Winner (UH-PRHLT-primary)**|76.70 ±0.00|
2016-test|cossim|76.45 ±10.40|0.25 ±0.04
2016-test|wmd-gensim|76.15 ±11.51|13.79 ±1.39
2016-test|**Baseline 1 (IR)**|74.75 ±0.00|
2016-test|wmd-relax|72.03 ±11.33|0.34 ±0.07
2016-test|**Baseline 2 (random)**|46.98 ±0.00|


Dataset | Strategy | MAP score | Elapsed time (sec)
:---|:---|:---|---:
2017-test|**Winner (SimBow-primary)**|47.22 ±0.00|
2017-test|wmd-relax|45.04 ±15.44|0.39 ±0.07
2017-test|cossim|44.38 ±14.71|0.29 ±0.05
2017-test|softcossim|44.25 ±15.68|4.89 ±0.80
2017-test|wmd-gensim|44.08 ±15.96|16.69 ±1.90
2017-test|**Baseline 1 (IR)**|41.85 ±0.00|
2017-test|**Baseline 2 (random)**|29.81 ±0.00|

## References

1. Grigori Sidorov et al. *Soft Similarity and Soft Cosine Measure: Similarity of Features in Vector Space Model*, 2014. ([link to PDF](http://www.scielo.org.mx/pdf/cys/v18n3/v18n3a7.pdf))
2. Delphine Charlet and Geraldine Damnati, SimBow at SemEval-2017 Task 3: Soft-Cosine Semantic Similarity between Questions for Community Question Answering, 2017. ([link to PDF](http://www.aclweb.org/anthology/S17-2051))
3. Thomas Mikolov et al. Efficient Estimation of Word Representations in Vector Space, 2013. ([link to PDF](https://arxiv.org/pdf/1301.3781.pdf))